# Analysis of the results

In [37]:
import os
import json
import numpy as np
from scipy import stats

## Get best results for each fold

In [38]:
def construct_path_from_parent(root_dir, subdir_list):
    # Construct the path using os.path.join
    return os.path.join(root_dir, *subdir_list)

In [39]:
root_dir = '/home/rob/Documents/3_projects/mac/_results/'
output_dir_base_path = '/home/rob/Documents/3_projects/mac/_analysis'
os.makedirs(output_dir_base_path, exist_ok=True)

current_dir = os.getcwd()
dataset_name = "kvasir"

model_names = ["vanilla", "mac"]

folds = [f'fold_{i}' for i in range(5)]
criteria = 'loss_dice'

if dataset_name in ["kvasir", "NucleiSeg", "URDE"]:
    perf_metrics = ['dice_score', 'IoU_score', 'precision_metric', 'recall_metric', 'accuracy_metric']
else:
    perf_metrics = ['IoU_score', 'precision_metric', 'recall_metric', 'accuracy_metric']

In [40]:
# def get_best_metrics(summary_loc: str, fold_nb: int, result_dict: dict, 
#                      perf_metrics: list, criteria: str, num_params: int) -> int:
#     with open(summary_loc) as file:
#         data = json.load(file)
#         
#     if num_params == 0:
#         num_params = data['n_params']
#     data = data['test']
# 
#     best_idx = np.argmin(data[criteria])
#     
#     fold_dict: dict = {}
#     for metric in perf_metrics:
#         fold_dict[metric] = data[metric][best_idx]
#     fold_dict['epoch'] = best_idx
#     
#     if fold_nb in result_dict:
#         raise KeyError(f"Key '{fold_nb}' already present in dictionary.")
#     
#     result_dict[fold_nb] = fold_dict
#     
#     return num_params

In [41]:
def get_best_metrics(summary_loc: str, fold_nb: int, result_dict: dict, 
                     perf_metrics: list, criteria: str, num_params: int) -> int:
    with open(summary_loc) as file:
        data = json.load(file)
        
    if num_params == 0:
        num_params = data.get('n_params', 0)
    data = data.get('test', {})

    if criteria not in data or not data[criteria]:
        raise ValueError(f"Criteria '{criteria}' is not in the data or has no values.")
    
    best_idx = np.argmin(data[criteria])
    
    if best_idx >= len(data[criteria]):
        raise IndexError("Calculated best index is out of bounds for the criteria list.")
    
    if not all(len(data[metric]) == len(data[criteria]) for metric in perf_metrics):
        raise ValueError("Mismatch in length between criteria and performance metrics.")
    
    fold_dict: dict = {}
    for metric in perf_metrics:
        fold_dict[metric] = data[metric][best_idx]
    fold_dict['epoch'] = best_idx
    
    if fold_nb in result_dict:
        raise KeyError(f"Key '{fold_nb}' already present in dictionary.")
    
    result_dict[fold_nb] = fold_dict
    
    return num_params


In [42]:
def build_path(dataset_name: str, 
               model_name: str, 
               fold_nb: str, 
               root_dir: str = '/home/rob/Documents/3_projects/bench/_results') -> str:
    subdirs = [dataset_name, model_name, fold_nb, "summary.json"]

    summary_loc = construct_path_from_parent(root_dir, subdirs)
    if os.path.exists(summary_loc):
        print(f'summary.json loc: {summary_loc}')
    else:
        print(f'No file at the path you provided: {summary_loc}.')
    
    return summary_loc

In [43]:
for model_name in model_names:
    result_dict: dict = {}
    num_params: int = 0
    
    for fold in folds:
        summary_loc = build_path(dataset_name, model_name, fold, root_dir=root_dir)
        num_params = get_best_metrics(summary_loc, fold, result_dict, perf_metrics, criteria, num_params)
        
    results = {'num_params': num_params}

    for metric in perf_metrics:
        data = np.array([result_dict[f"fold_{i}"][metric] for i in range(5)])
        mean = np.mean(data)
        ci = stats.t.interval(confidence=0.95, df=len(data)-1, loc=np.mean(data), scale=stats.sem(data))
        results[metric] = {
            "mean": mean,
            "95% CI": ci
        }
        
    output_path = os.path.join(output_dir_base_path, dataset_name)
    os.makedirs(output_path, exist_ok=True)
    
    json_name = f'/{dataset_name}_{model_name}_data.json'
    print(f'path to save: {output_path}{json_name}')
    with open(f'{output_path}{json_name}', 'w') as fp:
        json.dump(results, fp, default=str, indent=4)
        
        

summary.json loc: /home/rob/Documents/3_projects/mac/_results/kvasir/vanilla/fold_0/summary.json
summary.json loc: /home/rob/Documents/3_projects/mac/_results/kvasir/vanilla/fold_1/summary.json
summary.json loc: /home/rob/Documents/3_projects/mac/_results/kvasir/vanilla/fold_2/summary.json
summary.json loc: /home/rob/Documents/3_projects/mac/_results/kvasir/vanilla/fold_3/summary.json
summary.json loc: /home/rob/Documents/3_projects/mac/_results/kvasir/vanilla/fold_4/summary.json
path to save: /home/rob/Documents/3_projects/mac/_analysis/kvasir/kvasir_vanilla_data.json
summary.json loc: /home/rob/Documents/3_projects/mac/_results/kvasir/mac/fold_0/summary.json
summary.json loc: /home/rob/Documents/3_projects/mac/_results/kvasir/mac/fold_1/summary.json
summary.json loc: /home/rob/Documents/3_projects/mac/_results/kvasir/mac/fold_2/summary.json
summary.json loc: /home/rob/Documents/3_projects/mac/_results/kvasir/mac/fold_3/summary.json
summary.json loc: /home/rob/Documents/3_projects/mac